This notebook set's up an FEM simulation by sampling from the analytical approximation of the surface patch with some $\theta$. Then we translate this into a simnibs input matrix and run a simulation and return the results

In [2]:
import sys

In [3]:
#SCC package environment
sys.path.insert(0,'/KIMEL/tigrlab/projects/jjeyachandra/gmsh-sdk/lib/')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/miniconda2/envs/simnibs_env/lib/python2.7/site-packages')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/Python_modules/src')

In [4]:
import numpy as np
from simnibs import sim_struct, run_simulation

#For importing geometry library
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [5]:
#https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        
        
        path = os.path.join(self.path,fullname) + '.ipynb'

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [38]:
#Load up the geolibrary as a module
glib_path = '/projects/jjeyachandra/rtms_optimize/notebooks/python2/'
nbload = NotebookLoader(glib_path)
geolib = nbload.load_module('geolib')

importing Jupyter notebook from /projects/jjeyachandra/rtms_optimize/notebooks/python2/geolib.ipynb


In [39]:
#Load up quadratic surface constants, inverse rotation matrix, and bounds
C = np.fromfile('../../output/quadratic_vec')
iR = np.fromfile('../../output/inverse_rot').reshape((3,3))
b  = np.fromfile('../../output/param_bounds').reshape((3,2))

### Get raw input coordinates that need to be transformed

In [40]:
#Sample a coordinate within the bounds and form input array
x = np.random.uniform(b[0,0], b[0,1])
y = np.random.uniform(b[1,0], b[1,1])

#Sample rotation
r = np.random.uniform(0,180)
print(x,y,r)

(19.62572738592221, 30.78005873419909, 126.00370372507456)


In [41]:
#Compute z value then map to mesh surface
preaff_loc = geolib.map_param_2_surf(x,y,C)
loc = np.matmul(iR,preaff_loc)

#Compute rotation vector then align with mesh surface
preaff_rot = geolib.map_rot_2_surf(x,y,r,C)


In [44]:
preaff_rot

NameError: name 'preaff_rot' is not defined